# Text Summarization

![Text Summarization](https://blog.fpt-software.com/hs-fs/hubfs/image-8.png?width=376&name=image-8.png)

Text summarization คือ กระบวนการที่ใช้ในการสรุปข้อความยาวๆ ให้เป็นข้อความขนาดสั้นๆ ที่เข้าใจได้ง่าย และยังคงไว้ซึ่งสารที่ต้องการสื่อ

โดยทั่วไป การทำสรุปจะมีสองประเภท คือ การสรุปแบบ Extractive Summary และการสรุปแบบ Abstractive Summary

โดย Extractive Summary จะเป็นการเลือกประโยคที่มีใจความเด่นๆ ขึ้นมาทำเป็นสรุป
แต่ Abstractive Summary จะเป็นการเขียนข้อความขึ้นมาใหม่ให้สั้นและกระทัดรัด

# TF-IDF Summary in CNN Daily Mail

Term-frequency inverse-document-frequency (TF-IDF) เป็นลักษะณะเด่น หรือ feature แบบหนึ่งที่นิยมใช้กันมากในการทำ NLP

โดย feature นี้จะทำการสกัดเอาความถี่ของคำ (text frequency) ที่ปรากฏในเอกสาร (document) มาใช้ โดยสมมติฐานที่ว่า 
ยิ่งความถี่ของคำสูง ยิ่งเป็นลักษณะเด่นที่สามารถใช้เป็นตัวแทน (feature) ของ doc ได้ 

อย่างไรก็ตาม คำบางคำเป็นเป็นคำทั่วไป ที่ปรากฏอยู่มากในทุกเอกสาร ทำให้ไม่สารถนำมาใช้เป็นตัวแทนของข้อมูลได้ดีมากนัก จึงมีการใช้
ส่วนกลับของความถี่ของคำที่พบได้ทั่วไป (inverse document frequency) มาใช้งานร่วมด้วย

**โดย TF-IDF สามารถใช้เพื่อหาคำเด่นๆ เพื่อสร้างเป็นการสรุปบทความอย่างง่ายได้**

โดยในตัวอย่างนี้จะมีการแสดงขั้นตอนต่างๆ ดังนี้

1. Install and setup tools
2. Import modules
3. Load CNN-Daily Mail data
4. Create TF-IDF vectorizer
5. Transform text to TF-IDF
6. Create information for highlight visualization
7. Visualize highlight summary

## 1.Install tools 

[Attention Visualizer](https://github.com/abisee/attn_vis) เป็นเครื่องมือที่ใช้ในการแสดงผล highlight

In [1]:
!git clone https://github.com/abisee/attn_vis 2>/dev/null

## 2.Import modules

In [2]:
from pprint import pprint
import resource
import json

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from IPython.display import IFrame
from sklearn.feature_extraction.text import TfidfVectorizer
from rouge import Rouge

config open file limit

In [3]:
low, high = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (high, high))

## 3.Load CNN-Daily Mail data

Load CNN-Daily Mail data ด้วย Tensorflow datasets

In [4]:
train_dataset = tfds.load('cnn_dailymail', split='train', shuffle_files=False)

fetch ข้อมูลเข้า memory

In [5]:
ds_iterator = train_dataset.take(200).as_numpy_iterator()

example1 = ds_iterator.next()
example2 = ds_iterator.next()

แสดงตัวอย่าง data ใน dataset

In [6]:
print("Full article")
pprint(example1["article"].decode())

print("Abstractive highlight")
pprint(example1["highlights"].decode())

Full article
('By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . '
 'UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic '
 'Diocese in North Dakota has exposed potentially hundreds of church members '
 'in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late '
 'September and early October. The state Health Department has issued an '
 'advisory of exposure for anyone who attended five churches and took '
 'communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in '
 'North Dakota has exposed potentially hundreds of church members in Fargo, '
 'Grand Forks and Jamestown to the hepatitis A . State Immunization Program '
 "Manager Molly Howell says the risk is low, but officials feel it's important "
 'to alert people to the possible exposure. The diocese announced on Monday '
 'that Bishop John Folda is taking time off after being diagnosed with '
 'hepatitis A. The diocese says he contracted the infection through '
 '

## 4.Create TF-IDF vectorizer

Term-frequency inverse-document-frequency (TF-IDF) เป็นลักษะณะเด่น หรือ feature แบบหนึ่งที่นิยมใช้กันมากในการทำ NLP

โดย feature นี้จะทำการสกัดเอาความถี่ของคำ (text frequency) ที่ปรากฏในเอกสาร (document) มาใช้ โดยสมมติฐานที่ว่า 
ยิ่งความถี่ของคำสูง ยิ่งเป็นลักษณะเด่นที่สามารถใช้เป็นตัวแทน (feature) ของ doc ได้ 

อย่างไรก็ตาม คำบางคำเป็นเป็นคำทั่วไป ที่ปรากฏอยู่มากในทุกเอกสาร ทำให้ไม่สารถนำมาใช้เป็นตัวแทนของข้อมูลได้ดีมากนัก จึงมีการใช้
ส่วนกลับของความถี่ของคำที่พบได้ทั่วไป (inverse document frequency) มาใช้งานร่วมด้วย

**โดย TF-IDF สามารถใช้เพื่อหาคำเด่นๆ เพื่อสร้างเป็นการสรุปบทความอย่างง่ายได้**

สำหรับตัวอย่างนี้ จะทำการสร้าง TF-IDF Vectorizer ซึ่งจะถูกใช้เพื่อแปลงบทความเป็น TF-IDF ในรูปแบบ vector 1d (ต่อบทความ)

โดยจะเลือกใช้คำศัพท์ไม่เกิน 500 ค และใช้การตัดคำอย่างง่าย (แบ่งคำตามช่องว่าง)

In [7]:
max_vocab = 500  # คำ

tfidf_vectorizer = TfidfVectorizer(
    analyzer='word',
    stop_words='english',
    max_features=max_vocab
)

vectorizer ที่ได้จะเป็นการตัดคำแบบทั่วไป เราควรจะ fit vectorizer ให้เข้ากับ dataset ที่เราต้องการ

ทำการ load ข้อมูลเข้า memory เพื่อสร้างเป็น corpus และ highlight

In [8]:
training_iterator = train_dataset.take(200).as_numpy_iterator()

corpus = [ example["article"].decode() for example in training_iterator ]


In [9]:
training_iterator = train_dataset.take(200).as_numpy_iterator()     # reset iterator

target_highlights = [ example["highlights"].decode() for example in training_iterator ]


ทำการ fit vecorizer กับ corpus

In [10]:
tfidf_vectorizer.fit(corpus)

TfidfVectorizer(max_features=500, stop_words='english')

## 5.Transform text to TF-IDF

ใช้ vectorizer ที่ได้เพื่อแปลงข้อมูลประเภท text เป็น feature  ที่มีลักษณะข้อมูลเป็น decimal vector ที่สามารถใช้เพื่อทำ machine learning ได้

In [11]:
article_features = tfidf_vectorizer.transform(corpus).todense()

ทำการเลือกคำที่มีลักษณะเด่นที่สุดมา 200 คำต่อบทความเพื่อสร้างเป็น summary

In [12]:
# คำศัพท์ที่ vectorizer สกัดได้จาก corpus
feature_names = np.array(tfidf_vectorizer.get_feature_names())

# function สำหรับการเลือกคำศัพท์ที่เด่นที่สุดมา top_n คำ
def top_words(response, top_n=200):
    sorted_indices = np.argsort(response[0, :])[0, :-(top_n+1):-1]
    return feature_names[sorted_indices].tolist()[0]

In [13]:
top_200_word_summaries = [ " ".join(top_words(article_feature)) for article_feature in article_features ] 

ลองแสดงผลลัพท์การสรุปข้อความที่ได้จากวิธีการที่ใช้

In [14]:
top_200_word_summaries[0]

'october north members hundreds john 25 2013 est state pictured says manager food press officials announced health september possible late department monday 15 feel 11 14 early taking published updated took month people time france following general forced gave friday forces friends foreign free game friend future forward games young flight europe discovered does doesn dog doing don dr drug earlier east end evidence final experience face fact family fans getting far father federal felt fight germany ground given iraq human idea including india information injury inside instead international interview investigation involved island house james january job johnson joy judge june just killed king know known huge hours goal having going gone good got government great died group half hand happened hard head hospital heard hearing held help helped high history hit hiv holiday home hope different deal didn britain appeared area areas asked assault atletico attack attacks attorney australia aut

คำนวณ rouge metrics ที่ใช้วัดผลการทำสรุปบทความ

In [15]:
# try evaluate rouge metric of first example
hypo = top_200_word_summaries[0]
target = target_highlights[0]

rouge = Rouge()
rouges = rouge.get_scores(hypo, target)[0]
r1_val, r2_val, rl_val = rouges['rouge-1']["f"], rouges['rouge-2']["f"], rouges['rouge-l']["f"]

print('Target:')
print(target)
print('Prediction:')
print(hypo)

print(f"ROUGE-1 '{r1_val}'")
print(f"ROUGE-2 '{r2_val}'")
print(f"ROUGE-L '{rl_val}'")
print(f"ROUGE-AVG '{np.mean([r1_val, r2_val, rl_val])}'", '\n--\n')

Target:
Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed .
Prediction:
october north members hundreds john 25 2013 est state pictured says manager food press officials announced health september possible late department monday 15 feel 11 14 early taking published updated took month people time france following general forced gave friday forces friends foreign free game friend future forward games young flight europe discovered does doesn dog doing don dr drug earlier east end evidence final experience face fact family fans getting far father federal felt fight germany ground given iraq human idea including india information injury inside instead international interview investigation involved island house james january job johnson joy judge june just killed king know known huge hours goal having going gone good got gove

## 6.Create information for highlight visualization

เครื่องมือที่จำใช้ visualize คือ

ซึ่งต้องการ format ข้อมูลสำหรับการแสดงผลดังนี้

```json
{
    "article_lst": article_lst,     # 1d list N_ariticle_tokens
    "p_gens": prob_vocab,           # 2d array N_decoded_lst x 1
    "decoded_lst": decoded_lst,     # 1d list N_hypo_tokens
    "abstract_str": target,         # a string
    "attn_dists": dists             # Equivalence to emission matrix in HMM   # N_decoded_lst * tf_idf  
}
```

สร้างฟั่งชั่นที่ใช้ gen tokens (list of words) และ attention distribution จาก tf-idf ด้วยความเข้าใจ

In [16]:
N_used_word = np.sum(list(tfidf_vectorizer.vocabulary_.values()))

# extract sorted text tokens and attention distribution from tf-idf feature
def get_viz_info_tf_idf(article, article_feature, top_n=200):
    sorted_indices = np.argsort(article_feature[0, :])[0, :-(top_n+1):-1]
    
    # text tokens aka decoded list
    decoded_lst = feature_names[sorted_indices].tolist()[0]
    
    # prob distribution for
    dists = tfidf_vectorizer.transform(decoded_lst).todense().tolist()

    # prob from vocabulary
    p_gens = article_feature[0, sorted_indices].transpose().tolist()
    
    return p_gens, decoded_lst, np.vstack(dists).tolist()

format ข้อมูลที่ใช้แสดงผลและบันทึกเป็นไฟล์ไว้ให้ [Attention Visualizer](https://github.com/abisee/attn_vis) แสดงผล

In [17]:
article_lst = corpus[0].split(" ")  # simple tokenize
p_gens, decoded_lst, dists = get_viz_info_tf_idf(article_lst, article_features[0], 20)

In [18]:
att_viz_json = {
    "article_lst": article_lst,     # 1d list N_ariticle_tokens
    "p_gens": p_gens,      # 2d array N_decoded_lst x 1
    "decoded_lst": decoded_lst,     # 1d list N_hypo_tokens
    "abstract_str": target,         # a string
    "attn_dists": dists             # Equivalence to emission matrix in HMM   # N_decoded_lst * tf_idf  
}
with open('./attn_vis/attn_vis_data.json', 'w') as outfile:
    json.dump(att_viz_json, outfile)

## 7.Visualize highlight summary

run shell command ใน project directory
```shell
cd attn_vis
python3 -m http.server
```

In [19]:
IFrame(src='http://localhost:8000', width=900, height=400)